## **MASTERGIS: Composicion y mosaicos**

En esta lectura, aprenderemos sobre:

- Como realizar mosaicos espaciales.
- Como realizar un mosaico considerando imagenes Landsat 8 y Landsat 7.

<img src="https://user-images.githubusercontent.com/16768318/73668658-120f6f80-469e-11ea-9c31-84ef520817f2.jpg" align="right" width = 60%/>

In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"1//05P739font3A1CgYIARAAGAUSNwF-L9IrQLJR51eiG045wEQTaEY58dcZqNDMR9DX_Kkh_guasDrd8dvft2a7AXhfq1hBk20FCLA"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

In [0]:
import ee
ee.Initialize()

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

En general, la **composicion** se refiere al proceso de combinar imagenes superpuestas espacialmente en una sola imagen basada en una funcion de agregacion. El **mosaico** se refiere al proceso de ensamblar conjuntos de datos de imagenes **espacialmente** para producir una imagen espacialmente continua. En Earth Engine, estos terminos se usan indistintamente, aunque se admiten tanto la composicion como el mosaico. Por ejemplo, considere la tarea de **componer varias imágenes en la misma ubicacion**. El siguiente ejemplo demuestra cómo hacer un compuesto de valor máximo:

In [0]:
# Cargue los datos NAIP
naip2004_2012 = ee.ImageCollection('USDA/NAIP/DOQQ')\
                  .filterBounds(ee.Geometry.Point(-71.08841, 42.39823))\
                  .filterDate('2004-07-01', '2012-12-31')\
                  .select(['R', 'G', 'B'])

# Realize un "composite" temporal de las imagenes con una funcion de valor maximo.
composite = naip2004_2012.max()
center = [-71.12532, 42.3712]
mapdisplay(center, {'max value composite':composite.getMapId()},zoom_start=12)

Considere la necesidad de mosaico para cuatro diferentes escenas realizadas al mismo tiempo, pero en diferentes ubicaciones. El siguiente ejemplo demuestra que usando el metodo `imageCollection.mosaic()` puede solucionarlo:

In [0]:
roi = ee.Geometry.Rectangle(-71.17965, 42.35125, -71.08824, 42.40584)
mapdisplay(center,{'roi':roi.getInfo(), 'composite_01':composite.getMapId()}, zoom_start=12)

In [0]:
# Cargue los datos NAIP
naip2012 = ee.ImageCollection('USDA/NAIP/DOQQ')\
             .filterBounds(roi)\
             .filterDate('2012-01-01', '2012-12-31')

# Creamos un mosaico espacial y displayamos
mosaic = naip2012.mosaic()
center = [-71.12532, 42.3712]
mapdisplay(center,{'roi':roi.getInfo(), 'spatial mosaic':mosaic.getMapId()},zoom_start=12)

### **Ejercicio:**
**Crear un composite del año 2018, en la selva de Peru, considerando los datos de las imagenes Landsat. Automatizar el proceso mediante una funcion.**

In [0]:
#  Parametros globales
range_date = ['2018-01-01', '2018-12-31']
center = [-72.628,-10.609]
roi = ee.Geometry.Point(center)
l7_viz_params = {'bands':['B5','B4','B3'],'min':0,'max': 4000, 'gamma':1.2}
l8_viz_params = {'bands':['B6','B5','B4'],'min':0,'max': 4000, 'gamma':1.2}
l78_viz_params = {'bands':['SWIR','NIR','RED'],'min':0,'max': 4000, 'gamma':1.2}

Funcion para obtener el tiempo de cada imagen

In [0]:
from datetime import datetime as dt
def get_dates(ic):
  dates = ic.aggregate_array('system:time_start').getInfo()
  anonym = lambda x: dt.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S')
  return list(map(anonym, dates))

##### **Landsat 7 - sin mascara**

In [0]:
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
       .filterBounds(roi)\
       .filterDate(range_date[0], range_date[1])\
       .filterMetadata('CLOUD_COVER','less_than',20)

In [0]:
# Displaye los resultados
to_plot_median = l7.median().getMapId(l7_viz_params)
mapdisplay(center,{'median_l7_cloud':to_plot_median})

##### **Landsat 7 - con mascara**

In [0]:
def l7_maskcloud(image):
  qa = image.select('pixel_qa')
  my_mask = (1 << 3) + (1 << 5) + (1 << 7)
  mask = qa.bitwiseAnd(my_mask).eq(0)
  ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')  
  return image.select(["B.*"]).addBands(ndvi).updateMask(mask).copyProperties(image, ["system:time_start"])

In [0]:
l7_nocloud = l7.map(l7_maskcloud)
l7_nocloud_nvdi = l7_nocloud.qualityMosaic('NDVI')
l7_nocloud_getmapid = l7_nocloud_nvdi.getMapId(l7_viz_params)
mapdisplay(center,{'ndvi_l7_nocloud':l7_nocloud_getmapid,
                   'median_l7_cloud':to_plot_median})

##### **Landsat 8 - sin mascara**

In [0]:
l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
       .filterBounds(roi)\
       .filterDate(range_date[0], range_date[1])\
       .filterMetadata('CLOUD_COVER','less_than',20)

In [0]:
# Displaye los resultados
l8_to_plot_median = l8.median().getMapId(l8_viz_params)
mapdisplay(center,{'median_l8_cloud':l8_to_plot_median})

##### **Landsat 8 - con mascara**

In [0]:
def l8_maskcloud(image):
  qa = image.select('pixel_qa')
  my_mask = (1 << 3) + (1 << 5) + (1 << 7) + (1 << 9) + (1 << 10)
  mask = qa.bitwiseAnd(my_mask).eq(0)
  ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')  
  return image.select(["B.*"]).addBands(ndvi).updateMask(mask).copyProperties(image, ["system:time_start"])

In [0]:
l8_nocloud = l8.map(l8_maskcloud)
l8_nocloud_nvdi = l8_nocloud.qualityMosaic('NDVI')
l8_nocloud_getmapid = l8_nocloud_nvdi.getMapId(l8_viz_params)
mapdisplay(center,{'ndvi_l8_nocloud':l8_nocloud_getmapid,
                   'median_l8_cloud':l8_to_plot_median})

##### **Landsat 7 y 8 - con mascara**

In [0]:
def select_image_l8(img):
  return img.select(['B6','B5','B4','NDVI']).rename('SWIR','NIR','RED','NDVI')
def select_image_l7(img):
  return img.select(['B5','B4','B3','NDVI']).rename('SWIR','NIR','RED','NDVI')

l7_veg = l7_nocloud.map(select_image_l7)
l8_veg = l8_nocloud.map(select_image_l8)
l78 =l8_veg.merge(l7_veg)

In [0]:
l78_nocloud_nvdi = l78.qualityMosaic('NDVI')
l78_nocloud_median = l78.median()
l78_nocloud_getmapid_ndvi = l78_nocloud_nvdi.getMapId(l78_viz_params)
l78_nocloud_getmapid_median = l78_nocloud_median.getMapId(l78_viz_params)
mapdisplay(center,{'l78_nocloud_getmapid_ndvi':l78_nocloud_getmapid_ndvi,
                   'l78_nocloud_getmapid_median':l78_nocloud_getmapid_median})

### **Funcion para generar composites a partir de L7 y L8 - vegetacion**

In [0]:
def composite_l7l8_vegetation(init_date, last_date, roi, cloud_per, calendar, composite = 'median'):
  '''
  Crear un composite de vegetacion a partir de datos Landsat 8 y Landsat  7

  init_date: Fecha de inicio
  last_date: Fecha de fin
  roi: Ambito de estudio
  cloud_per: Porcentaje de nubes
  calendar: Lista numerica de dos elementos. Que indica los meses a filtrar.
  composite: Funcion utilizada para realizar el composite. Actualmente 
             disponible: mean, ndvi (max) y none (no composite).
  '''
  def l7_maskcloud(image):
    qa = image.select('pixel_qa')
    my_mask = (1 << 3) + (1 << 5) + (1 << 7)
    mask = qa.bitwiseAnd(my_mask).eq(0)
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')  
    return image.addBands(ndvi)\
                .select(['B5','B4','B3','NDVI'])\
                .rename('SWIR','NIR','RED','NDVI')\
                .updateMask(mask)\
                .copyProperties(image, ["system:time_start"])
  def l8_maskcloud(image):
    qa = image.select('pixel_qa')
    my_mask = (1 << 3) + (1 << 5) + (1 << 7) + (1 << 9) + (1 << 10)
    mask = qa.bitwiseAnd(my_mask).eq(0)
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')  
    return image.addBands(ndvi)\
                .select(['B6','B5','B4','NDVI'])\
                .rename('SWIR','NIR','RED','NDVI')\
                .updateMask(mask)\
                .copyProperties(image, ["system:time_start"])
  l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
        .filterBounds(roi)\
        .filterDate(init_date, last_date)\
        .filterMetadata('CLOUD_COVER','less_than',cloud_per)\
        .filter(ee.Filter.calendarRange(calendar[0], calendar[1],'month'))\
        .map(l7_maskcloud)
  l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
        .filterBounds(roi)\
        .filterDate(init_date, last_date)\
        .filterMetadata('CLOUD_COVER','less_than',cloud_per)\
        .filter(ee.Filter.calendarRange(calendar[0], calendar[1],'month'))\
        .map(l8_maskcloud)
  l78 =l8.merge(l7)
  if composite == 'median':
    return l78.median()
  elif composite == 'ndvi':
    return l78.qualityMosaic('NDVI')
  else:
    return l78

Correr **composite_l7l8_vegetation** para una area determinada

In [0]:
# ------------------------------
#  PARAMETROS GLOBALES
# ------------------------------
range_date = ['2019-01-01', '2019-12-31'] #fecha de inicio y fin
center = [-72.628,-10.609]
roi = ee.Geometry.Point(center) #Area de interes (seleccione el path and row)
calendar = [4,11] # Seleccionar imagenes de Abril a Noviembre
cloud_per = 20 # Maximo porcentaje de cobertura de nubes a considerar
composite = 'median' # Funcion para generar el composite (median, ndvi)

In [0]:
composite = composite_l7l8_vegetation(init_date=range_date[0],
                                      last_date=range_date[1],
                                      roi=roi,
                                      cloud_per=cloud_per,
                                      calendar=calendar,
                                      composite=composite)

In [63]:
l78_viz_params = {'bands':['SWIR','NIR','RED'],'min':0,'max': 4000, 'gamma':1.2}
mapdisplay(center,{'composite_median':composite.getMapId(l78_viz_params)})